# Roof-Counting A2Z Pipeline

This notebook runs **all five** training experiments (PPYOLOE X Fast + four RTMDet variants) and then performs inference + **Weighted-Boxes-Fusion** ensembling across their outputs.  

**Sections**  
1. Setup & Dependencies  
2. Imports  
3. Configuration (paths & hyperparams)  
4. Data Preparation (CSV → COCO + train/val split)  
5. Experiment List (six configs)  
6. Training Loop  
7. Inference & Single-Model JSON Exports  
8. Ensembling & Submission  


## 1. Setup & Dependencies

In [ ]:
!pip install mmcv-full mmdet mmyolo openmim ensemble-boxes pycocotools sklearn wandb
!mim install mmengine mmcv mmdet mmyolo


## 2. Imports

In [ ]:
import os, json, shutil
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image

import mmcv
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector, init_detector, inference_detector
from ensemble_boxes import weighted_boxes_fusion



## 3. Configuration

In [ ]:
CFG = {
    "HOME": "/notebooks/ZINDI",
    "CSV_TRAIN": "/notebooks/ZINDI/Train.csv",
    "CSV_TEST":  "/notebooks/ZINDI/Test.csv",
    "SAMPLE_SUB": "/notebooks/ZINDI/SampleSubmission.csv",
    "IMAGE_FOLDER": "/notebooks/ZINDI/Images",
    "WORK_DIR": "work_dirs",
    "COCO_DIR": "coco",
    "VAL_SPLIT": 0.1,
    # Inference thresholds
    "BOX_SMALL_FACTOR": 0.02,
    "ENSEMBLE_IOU_THR": 0.5,
    "SCORE_THR": 0.4
}
os.makedirs(CFG["WORK_DIR"], exist_ok=True)
os.makedirs(CFG["COCO_DIR"], exist_ok=True)


## 4. Data Preparation: CSV → COCO & Stratified Split

In [ ]:
def get_image_dimensions(path):
    with Image.open(path) as img:
        return img.width, img.height

def csv_to_coco(csv_file, image_folder):
    df = pd.read_csv(csv_file)
    # Expect columns: ImageId, XMin, YMin, XMax, YMax, ClassId
    cats = [{"id":1,"name":"Other"},{"id":2,"name":"Tin"},{"id":3,"name":"Thatch"}]
    img_ids = list(df["ImageId"].unique())
    images, annotations = [], []
    id_map = {img: idx+1 for idx, img in enumerate(img_ids)}
    ann_id = 1
    for img in img_ids:
        w,h = get_image_dimensions(f"{image_folder}/{img}.tif")
        images.append({"id": id_map[img], "file_name": f"{img}.tif", "width":w, "height":h})
    for _, row in df.iterrows():
        x, y, x2, y2 = row["XMin"], row["YMin"], row["XMax"], row["YMax"]
        w, h = x2-x, y2-y
        annotations.append({
            "id": ann_id,
            "image_id": id_map[row["ImageId"]],
            "category_id": int(row["ClassId"]),
            "bbox": [x, y, w, h],
            "area": w*h,
            "iscrowd": 0,
            "segmentation": [[x, y, x+w, y, x+w, y+h, x, y+h]]
        })
        ann_id += 1
    return {"images":images, "annotations":annotations, "categories":cats}

def stratified_split(coco, val_size=0.1, seed=42):
    pairs = [(anno["image_id"], anno["category_id"]) for anno in coco["annotations"]]
    train_ids, val_ids = train_test_split(pairs, test_size=val_size, random_state=seed, stratify=[c for _,c in pairs])
    train_img_ids = {i for i,_ in train_ids}
    val_img_ids   = {i for i,_ in val_ids}
    def filter_ids(ids):
        imgs = [img for img in coco["images"] if img["id"] in ids]
        ann  = [anno for anno in coco["annotations"] if anno["image_id"] in ids]
        return {"images":imgs, "annotations":ann, "categories":coco["categories"]}
    return filter_ids(train_img_ids), filter_ids(val_img_ids)

# Build & write COCO JSONs
coco_all = csv_to_coco(CFG["CSV_TRAIN"], CFG["IMAGE_FOLDER"])
train_coco, val_coco = stratified_split(coco_all, CFG["VAL_SPLIT"])
with open(f'{CFG["COCO_DIR"]}/instances_train.json','w') as f: json.dump(train_coco, f)
with open(f'{CFG["COCO_DIR"]}/instances_val.json','w')   as f: json.dump(val_coco,   f)


## 5. Experiment List

In [ ]:

# Define each of the six runs (five training )
EXPERIMENTS = [
    # YOLO
    {
      "name":"ppyoloe_x_fast",
      "mmdet_cfg":"mmyolo/configs/ppyoloe/ppyoloe_x_fast_8xb16-300e_coco.py",
      "batch":16, "max_epochs":300, "resume":False, "wandb":False, "lr":1e-3
    },
    # RTMDet L variants
    {"name":"rtmdet_l_fast","mmdet_cfg":"configs/rtmdet/rtmdet_l_syncbn_fast_8xb8-100e_coco.py","batch":8,"max_epochs":100,"resume":True,"wandb":True,"lr":4e-3},
    {"name":"rtmdet_l_lp","mmdet_cfg":"configs/rtmdet/rtmdet_l_syncbn_fast_8xb2-400e_coco.py","batch":2,"max_epochs":400,"resume":True,"wandb":False,"lr":1e-4},
    {"name":"rtmdet_x_fast","mmdet_cfg":"configs/rtmdet/rtmdet_x_syncbn_fast_8xb8-200e_coco.py","batch":8,"max_epochs":200,"resume":False,"wandb":True,"lr":1e-3},
    {"name":"rtmdet_l_mid","mmdet_cfg":"configs/rtmdet/rtmdet_l_syncbn_fast_8xb4-100e_coco.py","batch":4,"max_epochs":100,"resume":True,"wandb":False,"lr":4e-3}
]



## 6. Training Loop


In [ ]:
from omegaconf import OmegaConf

def train_experiment(exp):
    # load base config
    cfg = Config.fromfile(exp["mmdet_cfg"])
    # overrides
    cfg.data.train.ann_file = f"{CFG['COCO_DIR']}/instances_train.json"
    cfg.data.train.img_prefix = CFG["IMAGE_FOLDER"]
    cfg.data.val.ann_file   = f"{CFG['COCO_DIR']}/instances_val.json"
    cfg.data.val.img_prefix = CFG["IMAGE_FOLDER"]
    cfg.optimizer.lr = exp["lr"]
    cfg.lr_config by_epoch = True
    cfg.runner.max_epochs = exp["max_epochs"]
    cfg.data.samples_per_gpu = exp["batch"]
    cfg.work_dir = os.path.join(CFG["WORK_DIR"], exp["name"])
    cfg.load_from = cfg.load_from if exp["resume"] else None
    # wandb hook
    if exp["wandb"]:
        cfg.log_config.hooks.append({"type":"WandbLoggerHook", "init_kwargs":{"project":"zindi-roof"}})
    # train
    set_random_seed(0, deterministic=True)
    os.makedirs(cfg.work_dir, exist_ok=True)
    train_detector(
      cfg.model, 
      [build_dataset(cfg.data.train)],
      cfg, 
      distributed=False, 
      validate=True
    )

# loop
for exp in EXPERIMENTS:
    print(f"--- Running {exp['name']} ---")
    train_experiment(exp)



## 7. Inference & Single-Model JSON Exports


In [ ]:
def infer_and_save(exp):
    ckpt = os.path.join(CFG["WORK_DIR"], exp["name"], "latest.pth")
    model = init_detector(exp["mmdet_cfg"], ckpt, device="cuda:0")
    results = inference_detector(model, CFG["IMAGE_FOLDER"])
    # convert to COCO-style list of dicts
    dets = []
    for img_id, preds in enumerate(results, start=1):
        for cls_id, bboxes in enumerate(preds, start=1):
            for x1,y1,x2,y2,score in bboxes:
                dets.append({
                  "image_id":img_id,
                  "category_id":cls_id,
                  "bbox":[x1,y1,x2-x1,y2-y1],
                  "score": score
                })
    out_json = f"{CFG['WORK_DIR']}/{exp['name']}/results.json"
    with open(out_json,"w") as f: json.dump(dets,f)
    return out_json

json_files = [infer_and_save(exp) for exp in EXPERIMENTS]
print("Generated JSONs:", json_files)


## 8. Ensembling & Submission

In [ ]:
def filter_small_boxes(detections, factor):
    by_img = {}
    for d in detections:
        by_img.setdefault(d["image_id"], []).append(d)
    out=[]
    for img_id, ds in by_img.items():
        max_area = max([b["bbox"][2]*b["bbox"][3] for b in ds])
        out += [d for d in ds if d["bbox"][2]*d["bbox"][3] >= max_area*factor]
    return out

# gather per-image lists for WBF
all_model_boxes, all_model_scores, all_model_labels = {},{},{}
for fpath in json_files:
    dets = json.load(open(fpath))
    filt = filter_small_boxes(dets, CFG["BOX_SMALL_FACTOR"])
    tmp = {}
    for d in filt:
        img = d["image_id"]
        all_model_boxes.setdefault(img, []).append([[*d["bbox"]]])
        all_model_scores.setdefault(img, []).append([d["score"]])
        all_model_labels.setdefault(img, []).append([d["category_id"]-1])  # zero-based

# fuse per image
ensemble = []
for img_id in all_model_boxes:
    boxes, scores, labels = weighted_boxes_fusion(
        all_model_boxes[img_id],
        all_model_scores[img_id],
        all_model_labels[img_id],
        iou_thr=CFG["ENSEMBLE_IOU_THR"],
        skip_box_thr=CFG["SCORE_THR"]
    )
    for b, s, l in zip(boxes, scores, labels):
        ensemble.append({
          "image_id": img_id,
          "category_id": int(l+1),
          "bbox": [*b],
          "score": float(s)
        })

# pivot to submission
sub_df = pd.read_csv(CFG["SAMPLE_SUB"])
counts = pd.DataFrame(ensemble).groupby(["image_id","category_id"]).size().reset_index(name="count")
# Map image_id back to file name
id_to_name = {i+1:str(img)+".tif" for i,img in enumerate(pd.read_csv(CFG["CSV_TEST"])["ImageId"])}
counts["ImageId"] = counts["image_id"].map(id_to_name)
# fill sample submission
for _, row in counts.iterrows():
    col = f"{row['category_id']}"
    sub_df.loc[sub_df.ImageId==row.ImageId, col] = row["count"]
sub_df.to_csv("final_submission.csv", index=False)
